# Loop 58 Analysis: CV-LB Relationship and Strategy Assessment

**Goal**: Analyze the CV-LB relationship to determine the best path forward.

**Key Question**: Is the target (0.0347) mathematically reachable with current approaches?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# All submissions with CV and LB scores
submissions = [
    {'exp': 'exp_000', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.0123, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.0105, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.0104, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.0097, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.0093, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.0092, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.0090, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.0087, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.0085, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.0083, 'lb': 0.0877},
    {'exp': 'exp_041', 'cv': 0.0090, 'lb': 0.0932},
    {'exp': 'exp_042', 'cv': 0.0145, 'lb': 0.1147},
]

df = pd.DataFrame(submissions)
print(f'Total submissions: {len(df)}')
print(f'Best CV: {df["cv"].min():.4f} ({df.loc[df["cv"].idxmin(), "exp"]})')
print(f'Best LB: {df["lb"].min():.4f} ({df.loc[df["lb"].idxmin(), "exp"]})')
print(f'Target LB: 0.0347')

In [ ]:
# Fit linear regression: LB = slope * CV + intercept
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])

print(f'\n=== CV-LB Linear Regression ===')
print(f'LB = {slope:.4f} * CV + {intercept:.4f}')
print(f'R-squared: {r_value**2:.4f}')
print(f'P-value: {p_value:.2e}')
print(f'\nInterpretation:')
print(f'  - Intercept: {intercept:.4f} (extrapolation error when CV=0)')
print(f'  - Slope: {slope:.4f} (LB increases {slope:.2f}x faster than CV)')
print(f'  - R²: {r_value**2:.4f} (relationship is highly linear)')

# Calculate required CV to hit target
target_lb = 0.0347
required_cv = (target_lb - intercept) / slope
print(f'\n=== Target Analysis ===')
print(f'Target LB: {target_lb}')
print(f'Required CV to hit target: {required_cv:.6f}')
if required_cv < 0:
    print(f'  -> IMPOSSIBLE: Required CV is negative!')
    print(f'  -> The intercept ({intercept:.4f}) > target ({target_lb})')
    print(f'  -> Even with CV=0, predicted LB would be {intercept:.4f}')
else:
    print(f'  -> Achievable if we can get CV to {required_cv:.6f}')

In [ ]:
# Plot CV vs LB
plt.figure(figsize=(10, 6))
plt.scatter(df['cv'], df['lb'], s=100, c='blue', alpha=0.7, label='Submissions')

# Add regression line
cv_range = np.linspace(0, df['cv'].max() * 1.1, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}*CV + {intercept:.4f} (R²={r_value**2:.3f})')

# Add target line
plt.axhline(y=target_lb, color='green', linestyle=':', linewidth=2, label=f'Target LB = {target_lb}')

# Add intercept line
plt.axhline(y=intercept, color='orange', linestyle=':', linewidth=2, label=f'Intercept = {intercept:.4f}')

# Labels
for i, row in df.iterrows():
    plt.annotate(row['exp'], (row['cv'], row['lb']), fontsize=8, alpha=0.7)

plt.xlabel('CV Score (MSE)')
plt.ylabel('LB Score (MSE)')
plt.title('CV vs LB Relationship - All Submissions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=150)
plt.show()

print(f'\nPlot saved to /home/code/exploration/cv_lb_relationship.png')

In [ ]:
# Analyze the gap between intercept and target
print('\n=== GAP ANALYSIS ===')
print(f'Intercept (extrapolation error): {intercept:.4f}')
print(f'Target LB: {target_lb:.4f}')
print(f'Gap: {intercept - target_lb:.4f}')
print(f'Gap as % of target: {(intercept - target_lb) / target_lb * 100:.1f}%')

print('\n=== CRITICAL INSIGHT ===')
if intercept > target_lb:
    print(f'The intercept ({intercept:.4f}) is ABOVE the target ({target_lb}).')
    print(f'This means:')
    print(f'  1. Even with perfect CV=0, predicted LB would be {intercept:.4f}')
    print(f'  2. The target is MATHEMATICALLY UNREACHABLE with current approach')
    print(f'  3. We need to CHANGE THE CV-LB RELATIONSHIP, not just improve CV')
    print(f'\nStrategies to reduce the intercept:')
    print(f'  1. Uncertainty-weighted predictions (conservative on extrapolation)')
    print(f'  2. Bias correction (subtract a constant from predictions)')
    print(f'  3. Extrapolation detection features')
    print(f'  4. Domain-specific constraints that generalize')
else:
    print(f'The intercept ({intercept:.4f}) is BELOW the target ({target_lb}).')
    print(f'The target is achievable by improving CV.')

In [ ]:
# Analyze residuals to see if any submissions deviate from the line
df['predicted_lb'] = slope * df['cv'] + intercept
df['residual'] = df['lb'] - df['predicted_lb']
df['residual_pct'] = df['residual'] / df['lb'] * 100

print('\n=== RESIDUAL ANALYSIS ===')
print(df[['exp', 'cv', 'lb', 'predicted_lb', 'residual', 'residual_pct']].to_string())

print(f'\nMean absolute residual: {df["residual"].abs().mean():.4f}')
print(f'Max positive residual: {df["residual"].max():.4f} ({df.loc[df["residual"].idxmax(), "exp"]})')
print(f'Max negative residual: {df["residual"].min():.4f} ({df.loc[df["residual"].idxmin(), "exp"]})')

# Check if any submission significantly deviates from the line
outlier_threshold = 0.005  # 0.5% deviation
outliers = df[df['residual'].abs() > outlier_threshold]
if len(outliers) > 0:
    print(f'\nOutliers (residual > {outlier_threshold}):')
    print(outliers[['exp', 'cv', 'lb', 'residual']])
else:
    print(f'\nNo significant outliers - all submissions fall on the same line.')

In [ ]:
# What would it take to reach the target?
print('\n=== PATH TO TARGET ===')
print(f'Current best LB: {df["lb"].min():.4f}')
print(f'Target LB: {target_lb:.4f}')
print(f'Gap to target: {df["lb"].min() - target_lb:.4f}')

# Option 1: Improve CV (won't work if intercept > target)
print(f'\nOption 1: Improve CV')
if required_cv < 0:
    print(f'  -> IMPOSSIBLE: Required CV is negative ({required_cv:.6f})')
else:
    print(f'  -> Required CV: {required_cv:.6f}')
    print(f'  -> Current best CV: {df["cv"].min():.6f}')
    print(f'  -> Improvement needed: {(df["cv"].min() - required_cv) / df["cv"].min() * 100:.1f}%')

# Option 2: Reduce intercept
print(f'\nOption 2: Reduce intercept')
required_intercept = target_lb - slope * df['cv'].min()
print(f'  -> Current intercept: {intercept:.4f}')
print(f'  -> Required intercept (with best CV): {required_intercept:.4f}')
print(f'  -> Reduction needed: {intercept - required_intercept:.4f}')

# Option 3: Reduce slope
print(f'\nOption 3: Reduce slope')
required_slope = (target_lb - intercept) / df['cv'].min()
print(f'  -> Current slope: {slope:.4f}')
print(f'  -> Required slope (with current intercept): {required_slope:.4f}')
if required_slope < 0:
    print(f'  -> IMPOSSIBLE: Required slope is negative')

In [ ]:
# Summary and recommendations
print('\n' + '='*60)
print('SUMMARY AND RECOMMENDATIONS')
print('='*60)

print(f'\n1. CV-LB RELATIONSHIP:')
print(f'   LB = {slope:.2f} * CV + {intercept:.4f} (R² = {r_value**2:.3f})')
print(f'   All 13 submissions fall on this line with high R².')

print(f'\n2. THE PROBLEM:')
print(f'   Intercept ({intercept:.4f}) > Target ({target_lb})')
print(f'   Even with CV=0, predicted LB = {intercept:.4f}')
print(f'   The target is MATHEMATICALLY UNREACHABLE by improving CV alone.')

print(f'\n3. WHAT WE NEED:')
print(f'   To reach target {target_lb} with best CV {df["cv"].min():.4f}:')
print(f'   - Reduce intercept from {intercept:.4f} to {required_intercept:.4f}')
print(f'   - OR find an approach that changes the CV-LB relationship')

print(f'\n4. STRATEGIES TO TRY:')
print(f'   a) Uncertainty-weighted predictions:')
print(f'      - Use GP uncertainty to be conservative on extrapolation')
print(f'      - Blend toward population mean when uncertain')
print(f'   b) Bias correction:')
print(f'      - Subtract a constant from predictions')
print(f'      - Optimal correction: ~{intercept - target_lb:.4f}')
print(f'   c) Extrapolation detection:')
print(f'      - Add features measuring distance to training distribution')
print(f'      - Be conservative when extrapolating')
print(f'   d) Study top public kernels more carefully:')
print(f'      - They may have solved this problem')
print(f'      - Look for techniques that change the CV-LB relationship')

print(f'\n5. WHAT NOT TO DO:')
print(f'   - Keep optimizing CV with standard ML approaches')
print(f'   - Try more model architectures (MLP, LGBM, XGB, GP, etc.)')
print(f'   - These all fall on the same CV-LB line')